This is a very basic exercise on how to log images and see the bounding box using WandB.
How it simplifies a lot of stuff for us in vizualization.

I followed the tutorial here:
https://wandb.ai/stacey/yolo-drive/reports/Bounding-Boxes-for-Object-Detection--Vmlldzo4Nzg4MQ

Also find this(https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/290062) amazing discussion thread by https://www.kaggle.com/mpwolke to understand why this competition has a huge impact on environment.



<h1>Basic Imports

In [ ]:
import os
import ast
import pandas as pd
import numpy as np
import wandb
from PIL import Image

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wb_api")
wandb.login(key=secret_value_0)

<h1>Reading CSV files

In [ ]:
train_csv = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test_csv = pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")
train_csv.head(20)

<h1> Initializing WandB

In [ ]:
run = wandb.init(project="visualization4", name="coral-reef")

In [ ]:
# Checking the number of frames in each video/video_id
train_group = train_csv.groupby('video_id').count()['image_id']
train_group

In [ ]:
data = []
for i in train_group.keys():
    data.append([i, train_group[i]])
    
data

<h1> Bar graph to vizualize no of frames in each video

In [ ]:
table = wandb.Table(data=data, columns = ["video_id", "No of Frames"])
wandb.log({"my_bar_chart_id" : wandb.plot.bar(table, "video_id",
                               "No of Frames", title="Custom Bar Chart")})

In [ ]:
# this is the order in which my classes will be displayed
display_ids = {"coral" : 1}
# this is a revese map of the integer class id to the string class label
class_id_to_label = { int(v) : k for k, v in display_ids.items()}

In [ ]:
class_id_to_label

<h1> Defining the image reading function

In [ ]:
def bounding_boxes(filename, v_boxes):
    '''
    inputs: 
        filename: path to input image
        v_boxes: list of bounding boxes(dict) in format (x, y, width height)
    outputs:
        box_image: wandb image
    '''
    all_boxes = []
    image = Image.open(filename)
    for b_i, box in enumerate(v_boxes):
        # get coordinates and labels
        box_data = {"position" : {
          "minX" : box['x'], # x1
          "maxX" : box['x'] + box['width'], #x2
          "minY" : box['y'], # y1
          "maxY" : box['y'] + box['height']}, #y2
          "class_id" : 1, #Defining the label as 1, and rest of the background is 0.
          # optionally caption each box with its class and score
          "box_caption" : "%s" % ("coral"),
          "domain" : "pixel",
          "scores" : { "score" : 1}}
        all_boxes.append(box_data) #List of all boxes on a single image in a list

    # log to wandb: raw image, predictions, and dictionary of class labels for each class id
    box_image = wandb.Image(image, boxes = {"gt": {"box_data": all_boxes, "class_labels" : class_id_to_label}}, classes = [{"id": 0, "name": "none"}, {"id": 1, "name": "coral"}])
    return box_image

In [ ]:
my_data = []
#Path to base folder
base_folder_path = '../input/tensorflow-great-barrier-reef/train_images/video_0'
image_list = sorted(os.listdir(base_folder_path))
# Considering first 50 images for display
for i in range(50):
    bounding_box_dict = train_csv.loc[i].annotations
    bounding_box_d = ast.literal_eval(bounding_box_dict)
    if len(bounding_box_d)>0:
        if train_csv.loc[i].image_id.split('-')[-1]+'.jpg' in image_list:
            image_name = train_csv.loc[i].image_id.split('-')[-1]+'.jpg'
            sample_image_path = os.path.join(base_folder_path, image_name)
            wandb_image = bounding_boxes(sample_image_path, bounding_box_d)
            my_data.append([image_name, wandb_image])
        
columns= ["image_no", "image"]
test_table = wandb.Table(data=my_data, columns=columns)
run.log({"coral_reef_boxes": test_table})
run.finish()

In [ ]:
bounding_box_count = []
for row_index, row in train_csv.iterrows():
    bounding_box_count.append(len(ast.literal_eval(row.annotations)))

min_boxes = min(bounding_box_count)
max_boxes = max(bounding_box_count)
average_boxes = np.mean(bounding_box_count)



In [ ]:
print("The minimum number of boxes in any training frame is {}".format(min_boxes))
print("The maximum number of boxes in any training frame is {}".format(max_boxes))
print("The average number of boxes in any training frame is {}".format(average_boxes))